<a href="https://colab.research.google.com/github/hargurjeet/Suprise_Housing/blob/main/Suprise_Housing_Case_study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing the data

In [24]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.preprocessing import MinMaxScaler

In [4]:
input_url = 'https://raw.githubusercontent.com/hargurjeet/Suprise_Housing/main/train.csv'
raw_data = pd.read_csv(input_url)
raw_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [5]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [6]:
#removing columns with high missing values
pd.set_option('display.max_rows', None)
columns_with_high_missing_values = raw_data.columns[[((raw_data.isnull().sum()/len(raw_data))*100 > 40)]]
columns_with_high_missing_values

/usr/local/lib/python3.8/dist-packages/pandas/core/indexes/base.py:4616: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getitem(key)


Index(['Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature'], dtype='object')

In [7]:
print(len(raw_data.columns))
raw_data = raw_data.drop(labels=columns_with_high_missing_values, axis=1)
print(len(raw_data.columns))

81
76


In [8]:
X = raw_data.drop('SalePrice', axis=1)
y = raw_data['SalePrice']

In [9]:
y.shape

(1460,)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

## Imputation

In [11]:
## seprating numberical and categorical columns
X_train_numberic = X_train.select_dtypes(include=['number'])
X_train_cat = X_train.select_dtypes(include=['object'])

In [12]:
X_train_numberic = X_train.select_dtypes(include=['number'])
X_train_numberic.shape

(978, 37)

In [13]:
X_train_cat.shape

(978, 38)

In [14]:
X_train_numberic.shape

(978, 37)

In [ ]:
imp_median = SimpleImputer(missing_values=np.nan, strategy='median')
X_train_numberic = pd.DataFrame(imp_median.fit_transform(X_train_numberic), columns = X_train_numberic.columns)
X_train_numberic

In [16]:
regressor = RandomForestRegressor(n_estimators=100, max_depth=10)

In [17]:
#===========================================================================
# perform a scikit-learn Recursive Feature Elimination (RFE)
#===========================================================================
# here we want only one final feature, we do this to produce a ranking
n_features_to_select = 1
rfe = RFE(regressor, n_features_to_select=n_features_to_select)
rfe.fit(X_train_numberic, y_train)

RFE(estimator=RandomForestRegressor(max_depth=10), n_features_to_select=1)

In [ ]:
#===========================================================================
# now print out the features in order of ranking
#===========================================================================
feature_list = []
from operator import itemgetter
features = X_train_numberic.columns.to_list()
for x, y in (sorted(zip(rfe.ranking_ , features), key=itemgetter(0))):
    print(x, y)
    feature_list.append(y)

In [22]:
top_10_numberic_features = X_train_numberic[feature_list[:10]]

In [25]:
top_10_numberic_features.head()

,OverallQual,GrLivArea,TotalBsmtSF,YearBuilt,BsmtFinSF1,1stFlrSF,GarageArea,LotArea,GarageCars,YearRemodAdd
0,6.0,1054.0,936.0,1963.0,763.0,1054.0,480.0,8800.0,2.0,1963.0
1,5.0,1120.0,1120.0,2007.0,206.0,1120.0,0.0,8402.0,0.0,2007.0
2,7.0,1616.0,1616.0,2005.0,0.0,1616.0,834.0,8688.0,3.0,2005.0
3,5.0,1073.0,1073.0,1965.0,836.0,1073.0,515.0,10289.0,2.0,1965.0
4,5.0,1389.0,1389.0,1974.0,1071.0,1389.0,418.0,10010.0,2.0,1975.0


In [27]:
# scaling the data between 0 and 1
scaler = MinMaxScaler()
top_10_numberic_features_scaled = pd.DataFrame(scaler.fit_transform(top_10_numberic_features), columns = top_10_numberic_features.columns)
top_10_numberic_features_scaled.head()

,OverallQual,GrLivArea,TotalBsmtSF,YearBuilt,BsmtFinSF1,1stFlrSF,GarageArea,LotArea,GarageCars,YearRemodAdd
0,0.555556,0.135644,0.153191,0.659420,0.135188,0.165213,0.338505,0.035056,0.50,0.216667
1,0.444444,0.148078,0.183306,0.978261,0.036499,0.180358,0.000000,0.033195,0.00,0.950000
2,0.666667,0.241522,0.264484,0.963768,0.000000,0.294172,0.588152,0.034532,0.75,0.916667
3,0.444444,0.139224,0.175614,0.673913,0.148122,0.169573,0.363188,0.042015,0.50,0.250000
4,0.444444,0.198757,0.227332,0.739130,0.189759,0.242084,0.294781,0.040711,0.50,0.416667
